# CrewAI Agent System: Business Ideas to PRDs

**Objective:** Build a multi-agent system using CrewAI that translates business ideas into comprehensive Product Requirement Documents (PRDs). This system automates the entire workflow from Day 1 and Day 2 labs.

**Target Audience:** Software engineers attending the AI-Driven Software Engineering Program.

**Crew Members:**
- **Requirements Analyst**: Extracts user stories and personas from business ideas
- **Product Manager**: Synthesizes requirements into formal PRDs
- **Technical Architect**: Creates database schemas and ADRs
- **Quality Reviewer**: Validates outputs for completeness and quality

## 1. Setup and Dependencies

First, we'll install CrewAI and set up our environment with access to the course utilities.

In [ ]:
# Install crewai if not already installed
# !pip install -r requirements_crewai.txt -q

In [ ]:
import sys
import os
import json
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import warnings

warnings.filterwarnings('ignore')

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import save_artifact, load_artifact, clean_llm_output

# Load environment variables
load_dotenv()

# Verify API keys
if not os.getenv("OPENAI_API_KEY") and not os.getenv("GOOGLE_API_KEY"):
    print("ERROR: No API keys found. Please check your .env file.")

else:    print("✓ Environment configured successfully")

c:\Users\labadmin\Documents\AG-AISOFTDEV\.venv\Lib\site-packages\flaml\__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


✓ Environment configured successfully


## 2. Configure LLM for CrewAI

CrewAI supports multiple LLM providers through LangChain. We'll configure it to use either OpenAI or Google's models.

In [ ]:
# Configure LLM for CrewAI
if os.getenv("OPENAI_API_KEY"):
    llm = ChatOpenAI(
        model='gpt-4o-mini',
        temperature=0.3,
        api_key=os.getenv("OPENAI_API_KEY")
    )
    print("Using OpenAI GPT-4o-mini")
elif os.getenv("GOOGLE_API_KEY"):
    llm = ChatGoogleGenerativeAI(
        model='gemini-2.0-flash-exp',
        temperature=0.3,
        google_api_key=os.getenv("GOOGLE_API_KEY")
    )
    print("Using Google Gemini 2.0 Flash")
else:
    raise ValueError("No API key configured")

Using OpenAI GPT-4o


## 3. Define Specialized Agents

Each agent has a specific role in the PRD generation workflow, mirroring the Day 1 and Day 2 lab processes.

In [ ]:
# 1. Requirements Analyst Agent
requirements_analyst = Agent(
    role='Senior Requirements Analyst',
    goal='Analyze business problems and extract comprehensive user stories with acceptance criteria',
    backstory="""You are an experienced requirements analyst with 10+ years in software development.
    You excel at understanding business needs and translating them into structured user stories.
    You always ensure user stories follow the Agile format and include detailed acceptance criteria
    in Gherkin (Given/When/Then) format.""",
    llm=llm,
    verbose=True,
    allow_delegation=False
)

# 2. Product Manager Agent
product_manager = Agent(
    role='Senior Product Manager',
    goal='Create comprehensive PRDs that serve as the source of truth for development teams',
    backstory="""You are a seasoned product manager who has launched multiple successful products.
    You know how to synthesize requirements into clear, actionable PRDs that include all necessary
    sections: Introduction, User Personas, Features, Success Metrics, and Out of Scope items.
    You write in a professional, concise style that developers love.""",
    llm=llm,
    verbose=True,
    allow_delegation=False
)

# 3. Technical Architect Agent
technical_architect = Agent(
    role='Staff Software Engineer & Technical Architect',
    goal='Design scalable, normalized database schemas and document technical decisions',
    backstory="""You are a staff engineer with deep expertise in database design and system architecture.
    You follow best practices for normalization, use appropriate constraints and indexes,
    and always consider scalability. You document technical decisions using ADR format
    (Architectural Decision Records).""",
    llm=llm,
    verbose=True,
    allow_delegation=False
)

# 4. Quality Reviewer Agent
quality_reviewer = Agent(
    role='Senior QA Engineer & Documentation Specialist',
    goal='Ensure all artifacts meet quality standards and are complete',
    backstory="""You are a meticulous quality assurance engineer who reviews all deliverables
    for completeness, accuracy, and adherence to standards. You validate JSON structures,
    check that PRDs contain all required sections, verify SQL schemas are properly normalized,
    and ensure acceptance criteria are testable.""",
    llm=llm,
    verbose=True,
    allow_delegation=False
)

print("✓ All agents created successfully")

[autogen.oai.client: 10-31 14:21:21] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 10-31 14:21:22] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 10-31 14:21:22] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 10-31 14:21:22] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 10-31 14:21:22] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 10-31 14:21:22] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
✓ All agents initialized successfully
[autogen.oai.client: 10-31 14:21:22] {164} WARNING - The API key spe

## 4. Business Problem Input

Define the business problem that will be transformed into a PRD.

In [5]:
# You can modify this to test different business ideas
business_problem = """
We need a tool to help our company's new hires get up to speed. 
New employees often feel overwhelmed in their first weeks and don't know 
what tasks to complete or who to talk to. We want to create a system that 
guides them through onboarding, tracks their progress, and helps managers 
monitor completion.
"""

print("Business Problem Statement:")
print(business_problem)

Business Problem Statement:

We need a tool to help our company's new hires get up to speed. 
New employees often feel overwhelmed in their first weeks and don't know 
what tasks to complete or who to talk to. We want to create a system that 
guides them through onboarding, tracks their progress, and helps managers 
monitor completion.



## 5. Workflow 1: Generate User Stories

The first workflow focuses on requirements analysis - converting the business problem into structured user stories.

In [6]:
# Create a group chat for user story generation
user_story_groupchat = GroupChat(
    agents=[user_proxy, requirements_analyst, quality_reviewer],
    messages=[],
    max_round=8
)

user_story_manager = GroupChatManager(
    groupchat=user_story_groupchat, 
    llm_config=llm_config
)

# Initial message to start the workflow
user_story_task = f"""
TASK: Generate User Stories from Business Problem

Business Problem:
{business_problem}

RequirementsAnalyst: Please analyze this business problem and generate a comprehensive 
set of user stories in JSON format. Include at least 5 user stories with detailed 
acceptance criteria in Gherkin format.

QualityReviewer: After the RequirementsAnalyst provides the JSON, validate it for:
- Proper JSON structure
- All required fields present
- Acceptance criteria are specific and testable
- User stories follow the "As a...I want...so that" format

Once approved, save the JSON to 'artifacts/autogen_user_stories.json'.

When complete, respond with "WORKFLOW_COMPLETE".
"""

print("\n" + "="*80)
print("WORKFLOW 1: USER STORY GENERATION")
print("="*80 + "\n")

user_proxy.initiate_chat(
    user_story_manager,
    message=user_story_task
)

[autogen.oai.client: 10-31 14:21:23] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.

WORKFLOW 1: USER STORY GENERATION

Coordinator (to chat_manager):


TASK: Generate User Stories from Business Problem

Business Problem:

We need a tool to help our company's new hires get up to speed. 
New employees often feel overwhelmed in their first weeks and don't know 
what tasks to complete or who to talk to. We want to create a system that 
guides them through onboarding, tracks their progress, and helps managers 
monitor completion.


RequirementsAnalyst: Please analyze this business problem and generate a comprehensive 
set of user stories in JSON format. Include at least 5 user stories with detailed 
acceptance criteria in Gherkin format.

QualityReviewer: After the RequirementsAnalyst provides the JSON, validate it for:
- Proper JSON structure
- All required fields present
- Acceptance criteria are specific and testable
- Use

ChatResult(chat_id=None, chat_history=[{'content': '\nTASK: Generate User Stories from Business Problem\n\nBusiness Problem:\n\nWe need a tool to help our company\'s new hires get up to speed. \nNew employees often feel overwhelmed in their first weeks and don\'t know \nwhat tasks to complete or who to talk to. We want to create a system that \nguides them through onboarding, tracks their progress, and helps managers \nmonitor completion.\n\n\nRequirementsAnalyst: Please analyze this business problem and generate a comprehensive \nset of user stories in JSON format. Include at least 5 user stories with detailed \nacceptance criteria in Gherkin format.\n\nQualityReviewer: After the RequirementsAnalyst provides the JSON, validate it for:\n- Proper JSON structure\n- All required fields present\n- Acceptance criteria are specific and testable\n- User stories follow the "As a...I want...so that" format\n\nOnce approved, save the JSON to \'artifacts/autogen_user_stories.json\'.\n\nWhen compl

## 6. Extract and Save User Stories

Extract the generated user stories from the conversation and save them as an artifact.

In [7]:
# Extract user stories from the conversation
def extract_json_from_messages(messages):
    """Extract JSON content from agent messages."""
    for message in reversed(messages):
        content = message.get('content', '')
        
        # Try to find JSON in the message
        if '```json' in content:
            json_str = content.split('```json')[1].split('```')[0].strip()
        elif content.strip().startswith('['):
            json_str = content.strip()
        else:
            continue
            
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            continue
    
    return None

# Extract user stories
user_stories = extract_json_from_messages(user_story_groupchat.messages)

if user_stories:
    # Save to artifacts
    save_artifact(
        json.dumps(user_stories, indent=2), 
        "artifacts/autogen_user_stories.json",
        overwrite=True
    )
    print(f"✓ Saved {len(user_stories)} user stories to artifacts/autogen_user_stories.json")
    print("\nSample User Story:")
    print(json.dumps(user_stories[0], indent=2))
else:
    print("⚠ Could not extract user stories from conversation")
    user_stories = []

✓ Saved 5 user stories to artifacts/autogen_user_stories.json

Sample User Story:
{
  "id": 1,
  "persona": "New Hire",
  "user_story": "As a New Hire, I want to have a checklist of tasks to complete, so that I can efficiently manage my onboarding process.",
  "acceptance_criteria": [
    "Given I am a new hire, When I log into the onboarding system, Then I should see a checklist of tasks to complete.",
    "Given I have completed a task, When I mark it as complete, Then it should be updated in the system.",
    "Given I have tasks pending, When I view my checklist, Then I should see which tasks are incomplete."
  ]
}


## 7. Workflow 2: Generate PRD from User Stories

The second workflow synthesizes user stories into a formal Product Requirements Document.

In [8]:
# Load the PRD template
prd_template = load_artifact("prd_template.md", base_dir=os.path.join(project_root, "templates"))

if not prd_template:
    print("⚠ Warning: PRD template not found. Using default structure.")
    prd_template = """# Product Requirements Document

## Introduction
## User Personas
## Features / User Stories
## Success Metrics
## Out of Scope
"""

# Create a new group chat for PRD generation
prd_groupchat = GroupChat(
    agents=[user_proxy, product_manager, quality_reviewer],
    messages=[],
    max_round=10
)

prd_manager = GroupChatManager(
    groupchat=prd_groupchat, 
    llm_config=llm_config
)

# Prepare the PRD generation task
prd_task = f"""
TASK: Generate Product Requirements Document

User Stories (JSON):
{json.dumps(user_stories, indent=2)}

PRD Template:
{prd_template}

ProductManager: Please create a comprehensive PRD by:
1. Using the user stories as the foundation
2. Following the PRD template structure
3. Filling in all sections with relevant, professional content
4. Inferring reasonable content for Success Metrics and Out of Scope sections
5. Output the final PRD in clean markdown format

QualityReviewer: After the ProductManager provides the PRD, validate it for:
- All template sections are present and complete
- Content is professional and comprehensive
- User stories are properly incorporated
- Success metrics are measurable
- Out of Scope section is clear

Once approved, save the PRD to 'artifacts/autogen_prd.md'.

When complete, respond with "WORKFLOW_COMPLETE".
"""

print("\n" + "="*80)
print("WORKFLOW 2: PRD GENERATION")
print("="*80 + "\n")

user_proxy.initiate_chat(
    prd_manager,
    message=prd_task
)

[autogen.oai.client: 10-31 14:21:33] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.

WORKFLOW 2: PRD GENERATION

Coordinator (to chat_manager):


TASK: Generate Product Requirements Document

User Stories (JSON):
[
  {
    "id": 1,
    "persona": "New Hire",
    "user_story": "As a New Hire, I want to have a checklist of tasks to complete, so that I can efficiently manage my onboarding process.",
    "acceptance_criteria": [
      "Given I am a new hire, When I log into the onboarding system, Then I should see a checklist of tasks to complete.",
      "Given I have completed a task, When I mark it as complete, Then it should be updated in the system.",
      "Given I have tasks pending, When I view my checklist, Then I should see which tasks are incomplete."
    ]
  },
  {
    "id": 2,
    "persona": "New Hire",
    "user_story": "As a New Hire, I want to know who to contact for help, so that I can resolve my queries quic

ChatResult(chat_id=None, chat_history=[{'content': '\nTASK: Generate Product Requirements Document\n\nUser Stories (JSON):\n[\n  {\n    "id": 1,\n    "persona": "New Hire",\n    "user_story": "As a New Hire, I want to have a checklist of tasks to complete, so that I can efficiently manage my onboarding process.",\n    "acceptance_criteria": [\n      "Given I am a new hire, When I log into the onboarding system, Then I should see a checklist of tasks to complete.",\n      "Given I have completed a task, When I mark it as complete, Then it should be updated in the system.",\n      "Given I have tasks pending, When I view my checklist, Then I should see which tasks are incomplete."\n    ]\n  },\n  {\n    "id": 2,\n    "persona": "New Hire",\n    "user_story": "As a New Hire, I want to know who to contact for help, so that I can resolve my queries quickly.",\n    "acceptance_criteria": [\n      "Given I am a new hire, When I access the contact section, Then I should see a list of relevant 

## 8. Extract and Save PRD

Extract the generated PRD from the conversation and save it.

In [9]:
# Extract PRD from conversation
def extract_markdown_from_messages(messages, marker="# Product Requirements Document"):
    """Extract markdown content from agent messages."""
    for message in reversed(messages):
        content = message.get('content', '')
        
        # Look for markdown content
        if marker in content or "## Introduction" in content:
            # Clean up markdown fences if present
            if '```markdown' in content:
                content = content.split('```markdown')[1].split('```')[0].strip()
            elif '```' in content:
                content = content.split('```')[1].split('```')[0].strip()
            
            return content
    
    return None

# Extract PRD
prd_content = extract_markdown_from_messages(prd_groupchat.messages)

if prd_content:
    # Save to artifacts
    save_artifact(
        prd_content, 
        "artifacts/autogen_prd.md",
        overwrite=True
    )
    print("✓ Saved PRD to artifacts/autogen_prd.md")
    print("\nPRD Preview (first 500 characters):")
    print(prd_content[:500] + "...")
else:
    print("⚠ Could not extract PRD from conversation")

✓ Saved PRD to artifacts/autogen_prd.md

PRD Preview (first 500 characters):
# Product Requirements Document: Onboarding Management System

| Status | **Draft** |
| :--- | :--- |
| **Author** | Product Management Team |
| **Version** | 1.0 |
| **Last Updated** | October 25, 2023 |

## 1. Executive Summary & Vision
The Onboarding Management System is designed to streamline the onboarding process for new hires, managers, and HR administrators. By providing a structured checklist, relevant contacts, and progress tracking, the system aims to enhance the onboarding experience...


## 9. Workflow 3: Generate Database Schema

The third workflow creates a database schema based on the PRD.

In [10]:
# Create a new group chat for schema generation
schema_groupchat = GroupChat(
    agents=[user_proxy, technical_architect, quality_reviewer],
    messages=[],
    max_round=10
)

schema_manager = GroupChatManager(
    groupchat=schema_groupchat, 
    llm_config=llm_config
)

# Prepare schema generation task
schema_task = f"""
TASK: Generate SQL Database Schema

PRD Content:
{prd_content if prd_content else 'Use the user stories provided earlier'}

TechnicalArchitect: Please design a normalized SQL database schema that:
1. Supports all features described in the PRD
2. Uses proper primary keys, foreign keys, and constraints
3. Follows database normalization best practices
4. Includes appropriate indexes for common queries
5. Uses SQLite-compatible syntax

Minimum tables required:
- users (id, name, email, role, created_at)
- onboarding_tasks (id, title, description, due_date, status, user_id, created_at)

Output ONLY the SQL CREATE TABLE statements without markdown fences.

QualityReviewer: After the TechnicalArchitect provides the schema, validate:
- All required tables are present
- Foreign key relationships are correct
- Data types are appropriate
- Constraints are properly defined

Once approved, save the schema to 'artifacts/autogen_schema.sql'.

When complete, respond with "WORKFLOW_COMPLETE".
"""

print("\n" + "="*80)
print("WORKFLOW 3: DATABASE SCHEMA GENERATION")
print("="*80 + "\n")

user_proxy.initiate_chat(
    schema_manager,
    message=schema_task
)

[autogen.oai.client: 10-31 14:21:57] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.

WORKFLOW 3: DATABASE SCHEMA GENERATION

Coordinator (to chat_manager):


TASK: Generate SQL Database Schema

PRD Content:
# Product Requirements Document: Onboarding Management System

| Status | **Draft** |
| :--- | :--- |
| **Author** | Product Management Team |
| **Version** | 1.0 |
| **Last Updated** | October 25, 2023 |

## 1. Executive Summary & Vision
The Onboarding Management System is designed to streamline the onboarding process for new hires, managers, and HR administrators. By providing a structured checklist, relevant contacts, and progress tracking, the system aims to enhance the onboarding experience, reduce repetitive queries, and ensure timely integration of new employees. The ultimate vision is to create a seamless and efficient onboarding journey that fosters engagement and productivity from day one.

## 2. The Problem


ChatResult(chat_id=None, chat_history=[{'content': '\nTASK: Generate SQL Database Schema\n\nPRD Content:\n# Product Requirements Document: Onboarding Management System\n\n| Status | **Draft** |\n| :--- | :--- |\n| **Author** | Product Management Team |\n| **Version** | 1.0 |\n| **Last Updated** | October 25, 2023 |\n\n## 1. Executive Summary & Vision\nThe Onboarding Management System is designed to streamline the onboarding process for new hires, managers, and HR administrators. By providing a structured checklist, relevant contacts, and progress tracking, the system aims to enhance the onboarding experience, reduce repetitive queries, and ensure timely integration of new employees. The ultimate vision is to create a seamless and efficient onboarding journey that fosters engagement and productivity from day one.\n\n## 2. The Problem\n\n**2.1. Problem Statement:**\nNew hires currently face a fragmented and overwhelming onboarding experience, leading to decreased initial productivity and

## 10. Extract and Save Database Schema

Extract the SQL schema and save it as an artifact.

In [11]:
# Extract SQL from conversation
def extract_sql_from_messages(messages):
    """Extract SQL content from agent messages."""
    for message in reversed(messages):
        content = message.get('content', '')
        
        # Look for SQL content
        if 'CREATE TABLE' in content.upper():
            # Clean up using the helper function
            sql = clean_llm_output(content, language='sql')
            return sql
    
    return None

# Extract schema
schema_sql = extract_sql_from_messages(schema_groupchat.messages)

if schema_sql:
    # Save to artifacts
    save_artifact(
        schema_sql, 
        "artifacts/autogen_schema.sql",
        overwrite=True
    )
    print("✓ Saved database schema to artifacts/autogen_schema.sql")
    print("\nSchema Preview:")
    print(schema_sql[:400] + "..." if len(schema_sql) > 400 else schema_sql)
else:
    print("⚠ Could not extract schema from conversation")

✓ Saved database schema to artifacts/autogen_schema.sql

Schema Preview:
CREATE TABLE users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE,
    role TEXT NOT NULL CHECK (role IN ('New Hire', 'Manager', 'HR Administrator')),
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE onboarding_tasks (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    description TEXT,
    due_date DATE,
   ...


## 11. Summary and Generated Artifacts

Review all artifacts generated by the AutoGen agent system.

In [12]:
print("\n" + "="*80)
print("AUTOGEN PRD SYSTEM - EXECUTION SUMMARY")
print("="*80 + "\n")

artifacts = [
    ("User Stories (JSON)", "artifacts/autogen_user_stories.json", user_stories),
    ("Product Requirements Document", "artifacts/autogen_prd.md", prd_content),
    ("Database Schema (SQL)", "artifacts/autogen_schema.sql", schema_sql),
]

print("Generated Artifacts:")
print("-" * 80)

for name, path, content in artifacts:
    if content:
        status = "✓ Generated"
        if isinstance(content, list):
            detail = f"({len(content)} user stories)"
        elif isinstance(content, str):
            detail = f"({len(content)} characters)"
        else:
            detail = ""
    else:
        status = "✗ Failed"
        detail = ""
    
    print(f"{status:15} {name:40} {detail}")
    print(f"{'':15} → {path}")
    print()

print("="*80)
print("\nNext Steps:")
print("1. Review the generated artifacts in the 'artifacts' directory")
print("2. Use autogen_prd.md as input for API development (Day 3)")
print("3. Execute autogen_schema.sql to create your database")
print("4. Iterate on the business problem to generate new variations")
print("\n✓ AutoGen PRD System execution complete!")


AUTOGEN PRD SYSTEM - EXECUTION SUMMARY

Generated Artifacts:
--------------------------------------------------------------------------------
✓ Generated     User Stories (JSON)                      (5 user stories)
                → artifacts/autogen_user_stories.json

✓ Generated     Product Requirements Document            (5687 characters)
                → artifacts/autogen_prd.md

✓ Generated     Database Schema (SQL)                    (1371 characters)
                → artifacts/autogen_schema.sql


Next Steps:
1. Review the generated artifacts in the 'artifacts' directory
2. Use autogen_prd.md as input for API development (Day 3)
3. Execute autogen_schema.sql to create your database
4. Iterate on the business problem to generate new variations

✓ AutoGen PRD System execution complete!


## 12. Optional: Generate ADR for Database Choice

As a bonus, we can have the agents create an Architectural Decision Record.

In [13]:
# Create ADR generation workflow
adr_groupchat = GroupChat(
    agents=[user_proxy, technical_architect, quality_reviewer],
    messages=[],
    max_round=8
)

adr_manager = GroupChatManager(
    groupchat=adr_groupchat, 
    llm_config=llm_config
)

adr_task = """
TASK: Create Architectural Decision Record for Database Choice

TechnicalArchitect: Please create an ADR documenting the decision to use SQLite 
for this onboarding tool. Include:

1. Title: ADR 001: Database Technology Selection
2. Status: Accepted
3. Context: What factors influenced this decision
4. Decision: Use SQLite as the primary database
5. Consequences: Both positive and negative implications

Consider alternatives like PostgreSQL, MongoDB, or cloud-based solutions.

QualityReviewer: Validate the ADR is complete and well-reasoned.

Save to 'artifacts/autogen_adr_001.md' when approved.

Respond with "WORKFLOW_COMPLETE" when done.
"""

print("\n" + "="*80)
print("BONUS WORKFLOW: ADR GENERATION")
print("="*80 + "\n")

user_proxy.initiate_chat(
    adr_manager,
    message=adr_task
)

[autogen.oai.client: 10-31 14:22:12] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.

BONUS WORKFLOW: ADR GENERATION

Coordinator (to chat_manager):


TASK: Create Architectural Decision Record for Database Choice

TechnicalArchitect: Please create an ADR documenting the decision to use SQLite 
for this onboarding tool. Include:

1. Title: ADR 001: Database Technology Selection
2. Status: Accepted
3. Context: What factors influenced this decision
4. Decision: Use SQLite as the primary database
5. Consequences: Both positive and negative implications

Consider alternatives like PostgreSQL, MongoDB, or cloud-based solutions.

QualityReviewer: Validate the ADR is complete and well-reasoned.

Save to 'artifacts/autogen_adr_001.md' when approved.

Respond with "WORKFLOW_COMPLETE" when done.


--------------------------------------------------------------------------------

BONUS WORKFLOW: ADR GENERATION

Coordinator (to chat_ma

ChatResult(chat_id=None, chat_history=[{'content': '\nTASK: Create Architectural Decision Record for Database Choice\n\nTechnicalArchitect: Please create an ADR documenting the decision to use SQLite \nfor this onboarding tool. Include:\n\n1. Title: ADR 001: Database Technology Selection\n2. Status: Accepted\n3. Context: What factors influenced this decision\n4. Decision: Use SQLite as the primary database\n5. Consequences: Both positive and negative implications\n\nConsider alternatives like PostgreSQL, MongoDB, or cloud-based solutions.\n\nQualityReviewer: Validate the ADR is complete and well-reasoned.\n\nSave to \'artifacts/autogen_adr_001.md\' when approved.\n\nRespond with "WORKFLOW_COMPLETE" when done.\n', 'role': 'assistant', 'name': 'Coordinator'}, {'content': "# ADR 001: Database Technology Selection\n\n## Status\nAccepted\n\n## Context\nThe decision to select a database technology for the onboarding tool was influenced by several factors, including the project's scope, expec

## Lab Conclusion

Congratulations! You've built a sophisticated multi-agent system using Microsoft AutoGen that:

1. **Automated Requirements Gathering**: Transformed vague business ideas into structured user stories
2. **Generated Professional Documentation**: Created production-ready PRDs following templates
3. **Designed Technical Architecture**: Generated normalized database schemas
4. **Ensured Quality**: Used a reviewer agent to validate all outputs

### Key Achievements:

- **End-to-End Automation**: The entire Day 1 and Day 2 workflow is now automated
- **Agent Collaboration**: Multiple specialized agents worked together like a real team
- **Quality Assurance**: Built-in validation ensures reliable outputs
- **Reusable System**: Can be applied to any business problem, not just onboarding tools

### Key Takeaways:

1. **Agent Specialization**: Each agent has a clear role, mirroring real team structures
2. **Workflow Orchestration**: GroupChat enables complex multi-step processes
3. **Quality Gates**: The reviewer agent acts as a quality gate between stages
4. **Artifact Generation**: Structured outputs enable automation in downstream processes

### Extending This System:

Consider adding:
- **Research Agent**: Can fetch external information about similar products
- **Cost Estimator Agent**: Analyzes PRD and estimates development effort
- **Risk Analyzer Agent**: Identifies potential technical and business risks
- **Test Plan Generator Agent**: Creates test strategies from acceptance criteria

This demonstrates the power of AI-driven software engineering: what once took days of manual work can now be accomplished in minutes with higher consistency and quality!